In [1]:
# !unzip in_data/data-compressed.zip

Archive:  in_data/data-compressed.zip
replace __MACOSX/._lish-moa? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [2]:
#imports, need to only run this once
!pip3 install numpy 
!pip3 install pandas 
!pip3 install torch torchvision 
!pip3 install scikit-learn
!pip3 install xgboost
!pip3 install category_encoders

# Gradient Boosted Tree

In [8]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split

D_PATH = 'lish-moa/'

train_features = pd.read_csv(D_PATH + 'train_features.csv')
train_drug  = pd.read_csv(D_PATH + 'train_drug.csv')
test_features = pd.read_csv(D_PATH + 'test_features.csv')
train_targets_scored = pd.read_csv(D_PATH + 'train_targets_scored.csv' )
train_targets_nonscored = pd.read_csv(D_PATH + 'train_targets_nonscored.csv' )

tr_mask = train_features['cp_type']=='ctl_vehicle'
ts_mask = test_features['cp_type']=='ctl_vehicle'

# replace str to binary
train_features['cp_dose'] = train_features['cp_dose'].replace({'D1': 0, 'D2': 1})
train_features['cp_type'] = train_features['cp_type'].replace({'trt_cp': 0, 'ctl_vehicle': 1})
test_features['cp_dose'] = test_features['cp_dose'].replace({'D1': 0, 'D2': 1})
test_features['cp_type'] = test_features['cp_type'].replace({'trt_cp': 0, 'ctl_vehicle': 1})

# Exclude rows where cp_type is "ctl_vehicle"
X = train_features[~tr_mask]
y = train_targets_scored[~tr_mask]


#removing sig_id column 
X = X.iloc[:, 1:].to_numpy()
submission_X_test = test_features.iloc[:, 1:].to_numpy() #only for the final submission
y = y.iloc[:, 1:].to_numpy()

#we are going to fake making a train, dev and test set 
X, X_test, y, y_test = train_test_split(X, y, test_size=0.7, random_state=42)
X_test, X_dev, y_test, y_dev = train_test_split(X_test, y_test, test_size=0.1, random_state=42)

In [10]:
import os 
import sys
import numpy as np 
import pandas as pd 
from xgboost import XGBClassifier 
from sklearn.model_selection import KFold 
from category_encoders import CountEncoder 
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.multioutput import MultiOutputClassifier


n_folds = 5

xgb_classifier = MultiOutputClassifier(XGBClassifier())
model = Pipeline([ 
                  ('classify', xgb_classifier)])


kf = KFold(n_splits = n_folds)

oof_preds = np.zeros(y.shape)

oof_losses = []

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f'fold: {fold_idx}')
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model.fit(X_train, y_train)
    
    val_preds = model.predict_proba(X_val) # list of preds per class
    val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    oof_preds[val_idx] = val_preds
    
    loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
    print(f'loss: {loss}')

    oof_losses.append(loss)
    
    


fold: 0
loss: 0.02444403897806023
fold: 1
loss: 0.023796916861031938
fold: 2
loss: 0.024426657469277226
fold: 3
loss: 0.02453364071274442
fold: 4
loss: 0.02343127583924411


# Neural Network 